
# Linear Discriminant Analysis Using Julia 


In [1]:
# importing libraries 

using LinearAlgebra
using Statistics
using RDatasets
using BenchmarkTools
using CSV
using DataFrames
using Plots

In [2]:
# _class_cov 

function _class_cov(X , y, priors)
```
    Computes within-class covariance matrix
    
    input: 
    X : Predictor matrix 
    y: target values 
    priors: class priors
        
    output:
    cov_ : Weighted within-class covariance matrix
    
```
    classes = sort(unique(y))
    cov_ = zeros(size(X)[2], size(X)[2])
    
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        cov_ += (priors[idx])*(cov(Matrix(Xg)))
    end
    return(cov_)
end

_class_cov (generic function with 1 method)

In [3]:
# mean_col

function mean_col(X)
```
    computes mean of each column 
    
    input:
    X: Input data 
    
    output:
    mean_vec: mean value of each column
    
```
    mean_vec = [mean(c) for c in eachcol(X)]
    return(mean_vec)
end

mean_col (generic function with 1 method)

In [4]:
# _class_Mean

function _class_Mean(X,y)
```
    Computes class mean 
    
    input: 
    X: Input data
    y: target values
    
    output
    
    mean_ :  column wise mean for each class"(a matrix of shape(number of attributes , number of classes))"
    
```
    classes = sort(unique(y))
    
    mean_ = zeros(size(X)[2] ,length(classes) )
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        mean_val = [mean(c) for c in eachcol(Xg)]
        mean_[: ,idx] = mean_val
    end
    return(mean_)
end

_class_Mean (generic function with 1 method)

In [5]:
# _priors 

function _priors(X,y)
```
    Computes class priors 
    
    input: 
    X: Input data
    y: target values
    
    output:
    prior_ :  prior value for each class

```

    classes = sort(unique(y))
    
    prior_ = zeros(length(classes) )
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        prop = size(Xg)[1]/size(X)[1]
        prior_[idx] = prop
    end
    return(prior_)
end

_priors (generic function with 1 method)

In [6]:
function _make_cat(y)
```
    transform target variable into numerical categorical varible 
    
    input:
    y: target variable
    
    output:
    my_arr: numerical values corresponding to each categorical varible"(1:number of classes)"
    
```
    classes = sort(unique(y))
    
    labels = [x for x in range(1,length(classes))]
    
    my_arr = [0 for x in range(1, length(y))]
    
    for (id ,val) in enumerate(y)
        for (idx, class) in enumerate(classes)
            if val == class
                my_arr[id] = idx
            end
        end
    end
    return(my_arr)
end

_make_cat (generic function with 1 method)

In [7]:
# _solve_lsqr
```
    Computes a straightforward solution of the optimal decision rule based directly on the discriminant functions.
       using formula
    coef_ = inv"(sigma)""*"_class_Mean
    
    Note: dimensionality reduction is not supported with this solver
    
    input: 
    X : Predictor matrix 
    y: target values 
    priors: class priors
        
    output:
    coef_ : coefficient of discriminant functions
    intercept_ :  intercept of discriminant functions
    
    
```


function _solve_lsqr(X,y , priors::Union{Vector, Nothing}=nothing)
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]
    
    coef_ = inv(cov_)*mean_
    intercept_ =  -0.5 * diag(transpose(mean_)*coef_) + ln_priors
    
    return(coef_, intercept_)
end

_solve_lsqr (generic function with 2 methods)

In [8]:

function _solve_eigen(X,y ,priors::Union{Vector, Nothing}=nothing )

```
    Computes the optimal solution of the Rayleigh coefficient"(J(W))", "(basically the ratio of 
    between class scatter to within class scatter)" 
    "J(W) = \frac{W.T*Sb*W}{W.T*Sw*W}"
    
    This solver supports both classification and dimensionality reduction
    
    input: 
    X : Predictor matrix 
    y: target values 
    priors: class priors
        
    output:
    coef_ : coefficient of discriminant functions
    intercept_ :  intercept of discriminant functions
       
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]
    
    Sw = cov_  # within scatter
    St = cov(Matrix(X))  # total scatter
    Sb = St - Sw
    
    evals , evecs = eigen(Sb, Sw)
    
    coef_ = (transpose(mean_)*evecs)*transpose(evecs)
    intercept_ =  -0.5 * diag(transpose(mean_)*transpose(coef_)) + ln_priors
    
    return(coef_, intercept_)
end

_solve_eigen (generic function with 2 methods)

In [9]:
function _solve_SVD(X,y , tol ,priors::Union{Vector, Nothing}=nothing)
```
    SVD solver
    
    This solver supports both classification and dimensionality reduction
    
    input: 
    X : Predictor matrix 
    y: target values 
    tol: tolerance value for cut-off eigen value of diag matrix of SVD 
    priors: class priors
    
    output:
    pramas:
        intercept: intercept values of discriminant functions 
        coef: coef of discriminant functions
        rank: rank 
        scalings: transformation matrix"(W)"
    
    
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    n_samples = size(X)[1]
    classes = sort(unique(y))
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]


    Xc = []
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        Xg_mean = broadcast(- , Xg , transpose(mean_[:,idx]))
        push!(Xc , Matrix(Xg_mean))
    end 

    xbar_ = mean_col(X)

    Xc = vcat(Xc...)

    std_ =  std(Xc , dims=1)
    std_[std_.==0] .= 1

    fac = 1.0 / (size(X)[1] - length(classes))

    X_temp = sqrt(fac)*broadcast(/, Xc , std_)

    U, S, Vt = svd(X_temp)

    S_diag = S[S.>=tol]
    rank = length(S_diag)
    scalings_temp = broadcast(/, Vt[:, 1:rank] , transpose(std_))
    scalings = broadcast(/, scalings_temp , transpose(S[1:rank]))

    n_classes = length(classes)
    fac_2 = ifelse.( n_classes==1 , 1, 1/(n_classes-1))
    temp_1 = [sqrt(val) for val in (n_samples * priors *fac_2) ]
    temp_2 = broadcast(-, mean_ , xbar_)
    temp_3 = broadcast(* , transpose(temp_2) , temp_1)

    X_new = temp_3*scalings

    U, S, Vt = svd(X_new, full=false)

    rank = length(S[S.>=tol])
    scalings_ = scalings*Vt[:, 1:rank]

    coef = transpose(temp_2)*scalings_

    intercept_ = (-0.5)*sum(coef.*coef , dims=2) + ln_priors
    coef_ = coef*transpose(scalings_)
    intercept_ = intercept_ - coef_*xbar_
    
    
    if length(classes) ==2
        coef_ = coef_[2,:] - coef_[1,:]
        intercept_ = intercept_[2] -  intercept_[1]
    end
    
    params = Dict("intercept" => intercept_ , "coef" => coef_, "rank" => rank, "scalings" => scalings_)

    return(params)
end





_solve_SVD (generic function with 2 methods)

In [10]:


function _solve_moment(X,y , tol ,priors::Union{Vector, Nothing}=nothing)

```
    SVD solver, uses standard estimators of the mean and variance "(replicating R's solution)"
    
    This solver supports both classification and dimensionality reduction
    
    input: 
    X : Predictor matrix 
    y: target values 
    tol: tolerance value for cut-off eigen value of diag matrix of SVD 
    priors: class priors
    
    output:
    pramas:
        intercept: intercept values of discriminant functions 
        coef: coef of discriminant functions
        rank: rank 
        scalings:  transformation matrix"(W)"
```
    
    if priors !== nothing
        @assert round(sum(priors); digits=4) == 1
    end
    
    if priors === nothing 
        priors = _priors(X,y)
    end
    
    n_samples = size(X)[1]
    classes = sort(unique(y))
    mean_ = _class_Mean(X,y)
    cov_ = _class_cov(X , y, priors)
    ln_priors = [log(prior) for prior in priors]

    Xc = []
    for (idx, group) in enumerate(classes)
        Xg = X[y .== group, :]
        Xg_mean = broadcast(- , Xg , transpose(mean_[:,idx]))
        push!(Xc , Matrix(Xg_mean))
    end

    xbar_ = mean_col(X)

    Xc = vcat(Xc...)

    std_ =  std(Xc , dims=1)
    std_[std_.==0] .= 1

    fac = 1.0 / (size(X)[1] - length(classes))
    X_temp = sqrt(fac)*broadcast(/, Xc , std_)
    
    
    U, S, Vt = svd(X_temp)
    S_diag = S[S.>=tol]
    rank = length(S_diag)
    scalings_temp = broadcast(/, Vt[:, 1:rank] , transpose(std_))
    scalings = broadcast(/, scalings_temp , transpose(S[1:rank]))

    n_classes = length(classes)
    fac_2 = ifelse.( n_classes==1 , 1, 1/(n_classes-1))

    temp_1 = [sqrt(val) for val in (n_samples * priors *fac_2) ]
    temp_2 = broadcast(-, mean_ , xbar_)
    temp_3 = broadcast(* , transpose(temp_2) , temp_1)

    X_new = temp_3*scalings

    U, S, Vt = svd(X_new, full=false)
    rank  = length(S[S.>=tol])
    scalings_ = scalings*Vt[:, 1:rank]
    intercept_temp_1 = transpose(temp_2)*scalings_
    intercept_ = (-0.5)*sum(intercept_temp_1.*intercept_temp_1 , dims=2) + ln_priors
    intercept_temp_2 = intercept_temp_1*transpose(scalings_)
    intercept_ = intercept_ - intercept_temp_2*xbar_
    params = Dict("intercept" => intercept_ , "coef" => scalings_, "rank" => rank, "scalings" => scalings_)

    return(params)
end


_solve_moment (generic function with 2 methods)

# Test

## IRIS

In [69]:
iris = dataset("datasets", "iris")

X = iris[:,1:4]

labels = [0 for y in 1:150]
for i = 1:size(iris, 1)
    x = iris[:, 5][i]
    if x == "setosa"
        labels[i] = 1
    elseif x == "virginica"
        labels[i] = 2
    else
        labels[i] = 3
    end
end

y = labels;

In [12]:
intercept, coef = _solve_eigen(X,y)
intercept

3×4 Matrix{Float64}:
 23.5442  23.5879   -16.4306   -17.3984
 12.4458   3.68528   12.7665    21.0791
 15.6982   7.07251    5.21145    6.43423

In [72]:
tol = 0.0001
paramas_iris = _solve_SVD(X,y , tol)
paramas_iris["coef"]

3×4 Matrix{Float64}:
  6.31476  12.1393   -16.9464   -20.7701
 -4.78356  -7.76327   12.2508    17.7075
 -1.5312   -4.37604    4.69567    3.06259

In [52]:
tol = 0.0001
@benchmark pramas = _solve_SVD(X,y , tol)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  249.092 μs …   7.331 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     261.724 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   303.838 μs ± 324.936 μs  ┊ GC (mean ± σ):  5.93% ± 5.50%

  ▆█▇▅▄▄▃▃▂▂▂▂▁                   ▁                             ▂
  ██████████████████▇▇▇█▇▆▇▇▇▇▇▇████████▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▆▆▁▁▄▄▅ █
  249 μs        Histogram: log(frequency) by time        614 μs <

 Memory estimate: 208.69 KiB, allocs estimate: 1178.

In [56]:
proj = pramas["coef"]

make_normal(transpose(proj))

4×3 Matrix{Float64}:
  0.209815  -0.20457   -0.210479
  0.403343  -0.331998  -0.601533
 -0.563065   0.523907   0.645468
 -0.690109   0.757265   0.420984

# Python's result on IRIS LDA : 
### Coef_

[[  6.31475846,  12.13931718, -16.94642465, -20.77005459],
[ -1.53119919,  -4.37604348,   4.69566531,   3.06258539],
[ -4.78355927,  -7.7632737 ,  12.25075935,  17.7074692 ]]

In [15]:
pramas["intercept"]

3×1 Matrix{Float64}:
 -15.477836726795022
 -33.53768673957079
  -2.0219741537631464

### Intercept_
[-15.47783673,  -2.02197415, -33.53768674]

In [44]:
@benchmark p = _solve_moment(X,y , 0.0001)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  253.830 μs …   7.766 ms  ┊ GC (min … max): 0.00% … 87.20%
 Time  (median):     262.909 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   290.054 μs ± 301.630 μs  ┊ GC (mean ± σ):  6.31% ±  5.77%

  ▄██▆▄▄▄▃▁▁                                                    ▂
  ████████████▇▇▇▇▇▇▇███▇▇▆▅▆▅▆▅▅▆▆█████▇▇▅▄▅▅▅▅▄▄▄▅▅▃▄▄▁▄▁▃▅▄▅ █
  254 μs        Histogram: log(frequency) by time        471 μs <

 Memory estimate: 229.44 KiB, allocs estimate: 1223.

In [74]:
params_R_iris = _solve_moment(X,y ,
    0.0001)
params_R_iris["coef"]

4×2 Matrix{Float64}:
  0.829378   0.0241021
  1.53447    2.16452
 -2.20121   -0.931921
 -2.81046    2.83919

In [65]:
params_R_iris = _solve_moment(X,y , 0.0001)
proj = params_R_iris["coef"]
make_normal(proj)

4×2 Matrix{Float64}:
  0.208742   0.00653196
  0.386204   0.586611
 -0.554012  -0.252562
 -0.70735    0.769453

In [50]:
function make_normal(X)
    
    temp = X.*X
    temp = sum(temp , dims = 1)
    temp = .√(temp)
    temp = broadcast(/ , X , temp)
    return(temp)
end

make_normal (generic function with 1 method)

In [51]:
make_normal(proj)

4×2 Matrix{Float64}:
  0.208742   0.00653196
  0.386204   0.586611
 -0.554012  -0.252562
 -0.70735    0.769453

###  R's result on IRIS LDA

Coefficients of linear discriminants:
                    LD1         LD2
Sepal.Length  0.8293776  0.02410215
Sepal.Width   1.5344731  2.16452123
Petal.Length -2.2012117 -0.93192121
Petal.Width  -2.8104603  2.83918785


## Penguin

In [37]:
using CSV
using DataFrames

df = CSV.read("penguins_lter.csv" , DataFrame)
df = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)","Sex" ]]
df = dropmissing(df::AbstractDataFrame)
y = df[!,"Sex"]
labels = zeros(length(y))
for (idx,sex) in enumerate(y)
    if sex =="MALE"
        labels[idx] = 1
    end
end
y = labels
X = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]];


In [79]:
df = CSV.read("file1.csv" , DataFrame)

X = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]]

y = df[!,"Species"]
labels = zeros(length(y))
for (idx,specie) in enumerate(y)
    if specie =="Adelie Penguin (Pygoscelis adeliae)"
        labels[idx] = 1
    elseif specie == "Gentoo penguin (Pygoscelis papua)"
        labels[idx] = 2
    end
end
y = labels;


In [78]:
params_R_penguin = _solve_moment(X,y,
    0.0001 )
params_R_penguin["coef"]

4×2 Matrix{Float64}:
  0.0883267   -0.417871
 -1.0373      -0.0210049
  0.0861628    0.0134747
  0.00129952   0.00171144

In [77]:
params_R_penguin["coef"]

4×2 Matrix{Float64}:
  0.0883267   -0.417871
 -1.0373      -0.0210049
  0.0861628    0.0134747
  0.00129952   0.00171144

In [59]:
make_normal(params_R["coef"])

4×2 Matrix{Float64}:
  0.084554    -0.998213
 -0.992998    -0.0501766
  0.0824825    0.0321884
  0.00124401   0.00408829

In [22]:

4×2 Matrix{Float64}:
  0.0820607   -0.998165
 -0.991605    -0.052959
  0.0999184    0.0290805
  0.00133546   0.00413342


4×2 Matrix{Float64}:
  0.084554    -0.998213
 -0.992998    -0.0501766
  0.0824825    0.0321884
  0.00124401   0.00408829


Coefficients of linear discriminants:
                            LD1          LD2
Culmen.Length..mm.   0.08832666 -0.417870885
Culmen.Depth..mm.   -1.03730494 -0.021004854
Flipper.Length..mm.  0.08616282  0.013474680
Body.Mass..g.        0.00129952  0.001711436


Coefficients of linear discriminants:
                            LD1          LD2
Culmen.Length..mm.   0.08832666 -0.417870885
Culmen.Depth..mm.   -1.03730494 -0.021004854
Flipper.Length..mm.  0.08616282  0.013474680
Body.Mass..g.        0.00129952  0.001711436

P

LoadError: syntax: extra token "of" after end of expression

In [23]:
params_R = _solve_moment(X,y , 0.0001 )

Dict{String, Any} with 4 entries:
  "rank"      => 2
  "intercept" => [-16.7292; 32.1089; -56.9873;;]
  "coef"      => [0.0883267 -0.417871; -1.0373 -0.0210049; 0.0861628 0.0134747;…
  "scalings"  => [0.0883267 -0.417871; -1.0373 -0.0210049; 0.0861628 0.0134747;…

##  R's result on PENGUIN LDA

Coefficients of linear discriminants:
                           LD1
bill_length_mm     0.036231765
bill_depth_mm      0.751144512
flipper_length_mm -0.005785753
body_mass_g        0.001889799

In [80]:
tol = 0.0001
pramas_py_penguin = _solve_SVD(X,y , tol)
pramas_py_penguin["coef"]

3×4 Matrix{Float64}:
  1.06215    2.06466  -0.206102  -0.00755752
 -0.755773   3.38789  -0.268351  -0.0023679
  0.340615  -5.30055   0.443381   0.00708508

In [62]:
make_normal(pramas_py["coef"])

3×4 Matrix{Float64}:
  0.78832    0.31184   -0.369528  -0.711199
 -0.56093    0.511695  -0.481137  -0.222831
  0.252803  -0.800577   0.794956   0.66674

In [ ]:
array([[ 1.06214785e+00,  2.06466324e+00, -2.06101568e-01,
        -7.55752064e-03],
       [-7.55773193e-01,  3.38788801e+00, -2.68350550e-01,
        -2.36789997e-03],
       [ 3.40615434e-01, -5.30055439e+00,  4.43380810e-01,
         7.08507560e-03]])

In [38]:
df = CSV.read("file2.csv" , DataFrame)

X = df[!,["Culmen Length (mm)" , "Culmen Depth (mm)", "Flipper Length (mm)","Body Mass (g)"]]

y = df[!,"Sex"]

labels = zeros(length(y))
for (idx,sex) in enumerate(y)
    if sex =="MALE"
        labels[idx] = 1
    end
end
y = labels;


In [40]:
#params = _solve_SVD(X,y , 0.0001 )

@benchmark pramas = _solve_SVD(X,y , tol)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  268.830 μs …   8.314 ms  ┊ GC (min … max): 0.00% … 94.31%
 Time  (median):     307.122 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   378.798 μs ± 494.362 μs  ┊ GC (mean ± σ):  9.55% ±  7.02%

       █                                                         
  ▆█▇▆██▇▃▃▂▂▂▂▂▃▃▃▂▁▁▁▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  269 μs           Histogram: frequency by time          677 μs <

 Memory estimate: 316.94 KiB, allocs estimate: 997.

In [29]:
params["coef"]

4-element Vector{Float64}:
  0.09300704575251453
  1.8569412293410679
 -0.0172074752474619
  0.004687819990745008

## Python's  result on Penguins LDA

### Coef_

array([[ 0.09300705,  1.85694123, -0.01720748,  0.00468782]])

In [30]:
pramas_py["rank"]

2

In [31]:
pramas_py["intercept"]

3×1 Matrix{Float64}:
 -16.72920937332333
  32.10890197012843
 -56.98730763237397

### intercept_

array([52.19978716])

## MPG

In [45]:
mpg = dataset("ggplot2", "mpg")

X = mpg[!,["Displ", "Cyl"]]
y = _make_cat(mpg[!,"Drv"])


234-element Vector{Int64}:
 2
 2
 2
 2
 2
 2
 2
 1
 1
 1
 1
 1
 1
 ⋮
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2

In [46]:
@benchmark pramas_mpg = _solve_moment(X,y , 0.0000001 )

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  225.801 μs …   5.603 ms  ┊ GC (min … max): 0.00% … 91.93%
 Time  (median):     233.302 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   257.359 μs ± 281.639 μs  ┊ GC (mean ± σ):  6.35% ±  5.53%

  ▅██▆▄▄▄▃▂▁                                                    ▂
  ████████████▇▆▇▇▆▆▆▆▆▇██▇▆▆▅▅▅▆▆▄▆▆▇▇████▇▇▆▅▅▆▄▄▁▃▃▄▄▃▃▄▄▄▄▃ █
  226 μs        Histogram: log(frequency) by time        413 μs <

 Memory estimate: 208.50 KiB, allocs estimate: 1083.

In [34]:
pramas_mpg["coef"]

2×2 Matrix{Float64}:
 -1.52677   1.80351
  0.38846  -1.66271

### R's result 
Coefficients of linear discriminants:
             LD1       LD2
displ -1.5267671  1.803505
cyl    0.3884603 -1.662712